In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import os
import numpy as np
import PIL
import PIL.Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
import shutil
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LSTM, BatchNormalization, LeakyReLU, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from scipy.ndimage import zoom
from PIL import Image
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from tensorflow.keras.models import Sequential # for assembling a Neural Network model
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout # adding layers to the Neural Network model
# from tensorflow.keras.utils import plot_model # for plotting model diagram
from tensorflow.keras.optimizers import Adam # for model optimization 
from tensorflow.keras.preprocessing.image import ImageDataGenerator


Tensorflow/Keras: 2.12.0


In [ ]:
# Define the input and output directories
input_dir = '/content/drive/MyDrive/dataset_folder/fabrics'
output_dir = '/content/drive/MyDrive/Identify/dataset_folder'

In [ ]:
# Define a dictionary to map class names to integer labels
class_dict = {'Acrylic': 0, 'Artificial_fur': 1, 'Artificial_leather': 2, 'Blended': 3, 'Chenille': 4, 'Corduroy': 5, 'Cotton': 6}

# Iterate over all subdirectories in the input directory
for class_name in os.listdir(input_dir):
    class_dir_path = os.path.join(input_dir, class_name)
    # Create a directory for the class in the output directory
    class_output_dir = os.path.join(output_dir, class_name)
    os.makedirs(class_output_dir, exist_ok=True)
    
    # Initialize the image counter for the current class
    img_counter = 1
    
    # Iterate over all subdirectories in the current subdirectory
    for num_dir in os.listdir(class_dir_path):
        num_dir_path = os.path.join(class_dir_path, num_dir)
        
        # Iterate over all files in the subdirectory
        for filename in os.listdir(num_dir_path):
            if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png') or filename.endswith('.txt') or filename.endswith('.jfif'):
                # Load the image
                img_path = os.path.join(num_dir_path, filename)
                img = cv2.imread(img_path)

                # Check if the image is loaded correctly
                if img is None:
                    print(f"Error: Failed to load image {img_path}")
                    continue
                
                # Perform preprocessing operations such as resizing and normalization
                img = cv2.resize(img, (256, 256))
                img = img.astype('float32') / 255.0
                
                # Save the preprocessed image in the output directory with a unique filename
                output_filename = f"img_{img_counter}.png"
                output_path = os.path.join(class_output_dir, output_filename)
                cv2.imwrite(output_path, (img * 255).astype('uint8'))
                print(f'Saved {output_filename} in {class_name}')
                
                # Increment the image counter
                img_counter += 1


Saved img_1.png in Acrylic
Saved img_2.png in Acrylic
Saved img_3.png in Acrylic
Saved img_4.png in Acrylic
Error: Failed to load image /content/drive/MyDrive/dataset_folder/fabrics/Acrylic/66/tag.txt
Saved img_5.png in Acrylic
Saved img_6.png in Acrylic
Saved img_7.png in Acrylic
Saved img_8.png in Acrylic
Error: Failed to load image /content/drive/MyDrive/dataset_folder/fabrics/Acrylic/65/tag.txt
Saved img_9.png in Acrylic
Saved img_10.png in Acrylic
Saved img_11.png in Acrylic
Saved img_12.png in Acrylic
Error: Failed to load image /content/drive/MyDrive/dataset_folder/fabrics/Acrylic/822/tag.txt
Saved img_13.png in Acrylic
Saved img_14.png in Acrylic
Saved img_15.png in Acrylic
Saved img_16.png in Acrylic
Error: Failed to load image /content/drive/MyDrive/dataset_folder/fabrics/Acrylic/506/tag.txt
Saved img_17.png in Acrylic
Saved img_18.png in Acrylic
Saved img_19.png in Acrylic
Saved img_20.png in Acrylic
Error: Failed to load image /content/drive/MyDrive/dataset_folder/fabrics/A

In [ ]:
# Set the paths for your train and validation directories
train_dir = '/content/drive/MyDrive/Identify/Train'
val_dir = '/content/drive/MyDrive/Identify/Val'

# Create the output directories if they don't exist
for class_name in class_dict.keys():
    class_dir = os.path.join(train_dir, class_name)
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)
    class_dir = os.path.join(val_dir, class_name)
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)

# Set the percentage of images to use for training (rest will be used for validation)
train_percent = 0.8

# Iterate over all class folders
for class_name in class_dict.keys():
    class_dir = os.path.join(output_dir, class_name)
    # Get a list of all image files in the class directory
    image_files = [filename for filename in os.listdir(class_dir) if filename.endswith('.png')]
    # Shuffle the image files
    random.shuffle(image_files)
    # Split the image files into train and validation sets
    num_train = int(len(image_files) * train_percent)
    train_files = image_files[:num_train]
    val_files = image_files[num_train:]
    # Copy training images to the train directory for the current class
    for filename in train_files:
        src_path = os.path.join(class_dir, filename)
        dst_path = os.path.join(train_dir, class_name, filename)
        shutil.copyfile(src_path, dst_path)
    # Copy validation images to the val directory for the current class
    for filename in val_files:
        src_path = os.path.join(class_dir, filename)
        dst_path = os.path.join(val_dir, class_name, filename)
        shutil.copyfile(src_path, dst_path)

In [ ]:
import numpy as np 
import keras
from keras.layers import AveragePooling2D, Dropout 
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam, RMSprop
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.applications import VGG16
from keras.callbacks import ModelCheckpoint
from sklearn.utils import class_weight



In [ ]:
class_dict = {'Acrylic': 0, 'Artificial_fur': 1, 'Artificial_leather': 2, 'Blended': 3, 'Chenille': 4, 'Corduroy': 5, 'Cotton': 6}

img_width = 64
img_height = 64
channels = 3

# Load the dataset
X_train = [] 
y_train = []

In [ ]:
train_dir = '/content/drive/MyDrive/Identify/Train'
val_dir = '/content/drive/MyDrive/Identify/Val'

In [ ]:
train_data = ImageDataGenerator(rescale = 1.0/255, 
                                rotation_range = 180, 
                                vertical_flip = True, 
                                horizontal_flip = True, 
                                brightness_range = [0.5, 1.5], 
                                zoom_range = [1, 1.5])

In [ ]:
train_generator = train_data.flow_from_directory(directory = train_dir, 
                                                target_size = (224,224), 
                                                classes = list(class_dict.keys()), 
                                               batch_size = 70, 
                                              shuffle = True)

Found 2394 images belonging to 7 classes.


In [ ]:
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
class_weights = dict(enumerate(class_weights))

In [ ]:
test_data = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_data.flow_from_directory(directory = val_dir, 
                                              target_size = (224,224), 
                                              classes = list(class_dict.keys()), 
                                              batch_size = 50, 
                                              shuffle = False)

Found 603 images belonging to 7 classes.


In [ ]:
vgg16_model = VGG16()
x = vgg16_model.layers[-9].output
x = AveragePooling2D(pool_size = (2,2))(x)
x = Dropout(0.5)(x)
x = Flatten(name="flatten")(x)
x = Dense(128, activation = 'relu')(x)
x = Dropout(0.5)(x)
x = Dense(7, activation = 'softmax')(x)
model = Model(inputs = vgg16_model.input, outputs = x)

model.compile(optimizer = RMSprop(lr=0.0001), loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

553467096/553467096 [==============================] - 6s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:

print("\nTraining.....")
checkpoint = ModelCheckpoint(filepath = '/content/drive/MyDrive/new_fab_model.h5', 
                             monitor='val_accuracy', 
                             verbose=1, save_best_only=True, mode='max')

history = model.fit(x=train_generator,
                    steps_per_epoch=60,
                    validation_data=test_generator,
                    validation_steps=2,
                    epochs=1000,
                    verbose=1,
                    callbacks=[checkpoint],
                    class_weight=class_weights)


Training.....
Epoch 1/1000
 2/60 [>.............................] - ETA: 31:29 - loss: 5.9845 - accuracy: 0.0595  